In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm

class VGG16_CIFAR10(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16_CIFAR10, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Dropout(0.2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.Dropout(0.3)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Dropout(0.4)
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
      x = self.block1(x)
      x = self.block2(x)
      x = self.block3(x)
      x = torch.flatten(x, 1)
      x = self.classifier(x)
      return x


In [ ]:
mean = [0.4914, 0.4822, 0.4465]
std = [0.2470, 0.2435, 0.2616]

BATCH_SIZE = 128

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16_CIFAR10().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)


100%|██████████| 170M/170M [00:03<00:00, 48.4MB/s]


In [ ]:
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(trainloader, desc=f"Epoka {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100.*correct/total

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    test_acc = 100.*correct/total

    scheduler.step()

    print(f"Epoka {epoch+1}/{num_epochs} – Loss: {running_loss/len(trainloader):.3f} – Train Acc: {train_acc:.2f}% – Test Acc: {test_acc:.2f}%")

Epoka 1/30: 100%|██████████| 391/391 [00:28<00:00, 13.80it/s]


Epoka 1/30 – Loss: 1.869 – Train Acc: 30.32% – Test Acc: 42.47%


Epoka 2/30: 100%|██████████| 391/391 [00:27<00:00, 14.31it/s]


Epoka 2/30 – Loss: 1.460 – Train Acc: 45.80% – Test Acc: 52.95%


Epoka 3/30: 100%|██████████| 391/391 [00:27<00:00, 14.21it/s]


...

Epoka 28/30 – Loss: 0.563 – Train Acc: 80.14% – Test Acc: 82.17%


Epoka 29/30: 100%|██████████| 391/391 [00:28<00:00, 13.94it/s]


Epoka 29/30 – Loss: 0.559 – Train Acc: 80.45% – Test Acc: 82.49%


Epoka 30/30: 100%|██████████| 391/391 [00:28<00:00, 13.94it/s]


Epoka 30/30 – Loss: 0.550 – Train Acc: 80.56% – Test Acc: 82.36%


In [ ]:
extra_epochs = 30
start_epoch = 30
end_epoch = start_epoch + extra_epochs

for epoch in range(start_epoch, end_epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(trainloader, desc=f"Epoka {epoch+1}/{end_epoch}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100.*correct/total

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    test_acc = 100.*correct/total

    scheduler.step()

    print(f"Epoka {epoch+1}/{end_epoch} – Loss: {running_loss/len(trainloader):.3f} – Train Acc: {train_acc:.2f}% – Test Acc: {test_acc:.2f}%")

Epoka 31/60: 100%|██████████| 391/391 [00:26<00:00, 14.75it/s]


Epoka 31/60 – Loss: 0.541 – Train Acc: 81.10% – Test Acc: 82.80%


Epoka 32/60: 100%|██████████| 391/391 [00:26<00:00, 14.97it/s]


Epoka 32/60 – Loss: 0.537 – Train Acc: 81.20% – Test Acc: 82.89%


Epoka 33/60: 100%|██████████| 391/391 [00:27<00:00, 14.46it/s]


...

Epoka 58/60 – Loss: 0.417 – Train Acc: 85.43% – Test Acc: 86.01%


Epoka 59/60: 100%|██████████| 391/391 [00:26<00:00, 14.77it/s]


Epoka 59/60 – Loss: 0.412 – Train Acc: 85.62% – Test Acc: 85.70%


Epoka 60/60: 100%|██████████| 391/391 [00:26<00:00, 15.03it/s]


Epoka 60/60 – Loss: 0.412 – Train Acc: 85.52% – Test Acc: 85.93%


In [ ]:
extra_epochs = 20
start_epoch = 60
end_epoch = start_epoch + extra_epochs

for epoch in range(start_epoch, end_epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(trainloader, desc=f"Epoka {epoch+1}/{end_epoch}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100. * correct / total

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_acc = 100. * correct / total

    scheduler.step()

    print(f"Epoka {epoch+1}/{end_epoch} – Loss: {running_loss/len(trainloader):.3f} – Train Acc: {train_acc:.2f}% – Test Acc: {test_acc:.2f}%")

Epoka 61/80: 100%|██████████| 391/391 [00:27<00:00, 14.08it/s]


Epoka 61/80 – Loss: 0.411 – Train Acc: 85.46% – Test Acc: 86.00%


Epoka 62/80: 100%|██████████| 391/391 [00:27<00:00, 14.15it/s]


Epoka 62/80 – Loss: 0.409 – Train Acc: 85.74% – Test Acc: 85.95%


Epoka 63/80: 100%|██████████| 391/391 [00:27<00:00, 14.39it/s]


...

Epoka 78/80 – Loss: 0.380 – Train Acc: 86.59% – Test Acc: 86.86%


Epoka 79/80: 100%|██████████| 391/391 [00:27<00:00, 14.24it/s]


Epoka 79/80 – Loss: 0.381 – Train Acc: 86.71% – Test Acc: 86.77%


Epoka 80/80: 100%|██████████| 391/391 [00:27<00:00, 14.41it/s]


Epoka 80/80 – Loss: 0.378 – Train Acc: 86.86% – Test Acc: 86.64%


In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, 'checkpoint_80_epoch_vgg.pth')


from google.colab import files
files.download('checkpoint_80_epoch_vgg.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>